# Maximum Finding

Maximum finding algorithm using qiskit library using n-bit comparator with minimum anciallary register used. 4-bit version will be experimented using the real device from IBM.

## The outline of algorithm

We will use n-qbit register to store our current maximum value called `qc`. In order to find the maximum value we need to repeatedly find one larger value (index) than `qc` and stop when we can't find any. For each iteration we will use `qs` to store our larger than `qc` value. 

0. randomly initialize `qc` and measure its value
1. initialize `qs` into super position of all states 
2. apply Grover's algorithm using 3-bit comparator as the black-box function comparing `qs` and `qc` and set the input search space on `qs`
3. if `qs` is larger than `qc` replace `qc` with `qs` value and repeat step (2)
4. if `qs` is not larger than `qc` then we conclude that `qs` is the maximum value

To start we need to import all the packages we need.

In [ ]:
# importing QISKit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, QISKitError
from qiskit import available_backends, execute, register, get_backend, compile

from qiskit.tools import visualization
from qiskit.tools.visualization import circuit_drawer

# import other necessary stuff
import random
import math
import warnings

import sys, time, getpass
try:
    sys.path.append("../")
    import Qconfig
    qx_config = {
        "APItoken": Qconfig.APItoken,
        "url": Qconfig.config['url']}
    print('Qconfig loaded from %s.' % Qconfig.__file__)
except:
    print('Qconfig.py not found in parent directory')

To find the maximum value we need comparator first. Since we will be using multiple qubit control-cot gate, we will make a function `n_control_not()` which we will use a lot later on.

In [ ]:
def n_control_not(circuit, q_array, target, ancillary_array, flip_array=None):
    # error checking
    if flip_array is not None and len(q_array) != len(flip_array):
        raise ValueError("q_array(len:{}) and flip_array(len:{}) must have the same length".format(len(q_array), len(flip_array)))
    if len(ancillary_array) < len(q_array) - 2:
        raise ValueError("ancillary array length is not enough ({}) for q_array ({})".format(len(ancillary_array), len(q_array)))
    if flip_array is None:
        flip_array = [1 for _ in range(len(q_array))]
    # todo handle wrong flip_array
    
    n = len(q_array)
    
    # put X-gate if flip
    for i in range(n):
        if flip_array[i] == -1:
            circuit.x(q_array[i])
        
    # special case for only 2 bits
    if n == 2:
        circuit.ccx(q_array[0], q_array[1], target)
    else:
        circuit.ccx(q_array[0], q_array[1], ancillary_array[0])
        for i in range(2, n-1):
            circuit.ccx(q_array[i], ancillary_array[i-2], ancillary_array[i-1])
        circuit.ccx(q_array[n-1], ancillary_array[n-3], target)
        for i in reversed(range(2, n-1)):
            circuit.ccx(q_array[i], ancillary_array[i-2], ancillary_array[i-1])
        circuit.ccx(q_array[0], q_array[1], ancillary_array[0])
    
    # clean up X-gate
    for i in range(n):
        if flip_array[i] == -1:
            circuit.x(q_array[i])

We already have the n_control_not gate so now, onto the comparator.

Let's test out n_control_not function we just implemented
with 2 qubits (the special case).

In [ ]:
qc = QuantumCircuit()
qs = QuantumRegister(2, 'qs')
anc = QuantumRegister(1, 'anc')
target = QuantumRegister(1, 'target')
qc.add(qs)
qc.add(anc)
qc.add(target)


n_control_not(qc, qs, target[0], anc)

circuit_drawer(qc)

 We will see that the ancillary register shouldn't be used as intended.
 
 And here's the 4 qubits version

In [ ]:
qc = QuantumCircuit()
qs = QuantumRegister(4, 'qs')
anc = QuantumRegister(2, 'anc')
target = QuantumRegister(1, 'target')
qc.add(qs)
qc.add(anc)
qc.add(target)


n_control_not(qc, qs, target[0], anc)

circuit_drawer(qc)

In [ ]:
def n_bit_comparator(qc, a, b, anc, target):
    
    # check whether two register is the same length
    if len(a) != len(b):
        raise ValueError('two register to compare must have the same length: a is {}, b is {}'.format(len(a), len(b)))
    
    anc_len = len(anc)
    n = len(a)
    if anc_len < 2 * n - 2:
        raise ValueError('ancillary bit is not enough: anc_len is {}, need at least {} qbit'.format(anc_len, len(a) - 1))
    
    # compare the MSB
    n_control_not(qc, [a[0], b[0]], target, anc, [1, -1])
    
    for i in range(1, n):
        # all more significant bits must be equal
        j = i-1
        qc.ccx(a[j], b[j], anc[j])
        qc.x(a[j])
        qc.x(b[j])
        qc.ccx(a[j], b[j], anc[j])
        
        n_control_not(qc, [anc[j] for j in range(i)] + [a[i], b[i]], target, [anc[k] for k in range(i, anc_len)],
                     [1] * (i+1) + [-1])
        
    for j in reversed(range(i)):
        qc.ccx(a[j], b[j], anc[j])
        qc.x(a[j])
        qc.x(b[j])
        qc.ccx(a[j], b[j], anc[j])

Let's see how the 4-qubit comparator looks like.

In [ ]:
n = 4

qc = QuantumCircuit()
a = QuantumRegister(n, 'a')
b = QuantumRegister(n, 'b')
anc = QuantumRegister(2 * n - 2, 'anc')
target = QuantumRegister(1, 'target')
qc.add(a)
qc.add(b)
qc.add(anc)
qc.add(target)

n_bit_comparator(qc, a, b, anc, target[0])

circuit_drawer(qc)

Before we can do some comparison testing, we implement some helper functions to help us out.

The `int_to_qubit` function will help us initializing qregs via binary string.

In [ ]:
def int_to_qubit(n, circuit, qs):
    bits_required = int(math.log(max(n,1), 2) + 1)
    qs_len = len(qs)
    if qs_len < bits_required:
        raise ValueError('input n = {} requires {} bits but qs is only {} bits'.format(n, bits_required, qs_len))

    bstr = '{0:b}'.format(n)
    bstr = bstr.zfill(qs_len)
    for i in range(qs_len):
        if bstr[i] == '1':
#             print('bit {} of {} is 1'.format(i, bstr))
            circuit.x(qs[i])

Let's test out this comparator a bit. By comparing 11 and 0 to 15.

In [ ]:
n = 4

for i in range(16):
    qc = QuantumCircuit()
    a = QuantumRegister(n, 'a')
    b = QuantumRegister(n, 'b')
    anc = QuantumRegister(2 * n - 2, 'anc')
    target = QuantumRegister(1, 'target')
    cs = ClassicalRegister(1, 'result')
    
    qc.add(a)
    qc.add(b)
    qc.add(anc)
    qc.add(target)
    qc.add(cs)
    
    int_to_qubit(11, qc, a)
    int_to_qubit(i, qc, b)
    
    n_bit_comparator(qc, a, b, anc, target[0])
    qc.measure(target, cs)
  
    circuit_drawer(qc)

    # Execute circuit
    job = execute([qc], backend='local_qasm_simulator', shots=10)
    result = job.result()

    counts = result.get_counts(qc)
    if i < 11 and counts['1'] == 10:
        print('correct: 11 > {}'.format(i))
    elif i >= 11 and counts['0'] == 10:
        print('correct: 11 <= {}'.format(i))
    else:
        print('error: 11 is neither larger nor less than {}'.format(i))
    
print('.....finished.....')

The next step is use the comparator and do Grover's search to find a larger value. We'll be using Boyer et al version of Grover's algorithm which goes as follow.

1. initialize $m = 1$ and set $\lambda = 6/5$
2. choose `j` uniformly at random among the nonnegative integers smaller than `m`
3. apply j iterations of Grover's algorithm starting from initial state $\left|\Psi_0\right\rangle = \sum_i \frac{1}{\sqrt{N}}\left|i\right\rangle$
4. observe the register: let $i$ be the outcome
5. if `T[i] = x`, the problem is solved: **exit**
6. otherwise, set `m` to min($\lambda m$, $\sqrt{N}$)    
and go back to step 2.

We will need inverse around the mean function

In [ ]:
def inv_around_mean(circuit, qs, anc):
    n = len(qs)
    for i in range(n):
        circuit.h(qs[i])
    for i in range(n):
        circuit.x(qs[i])
    circuit.h(qs[0])
    n_control_not(circuit, [qs[i] for i in range(1, n)], qs[0], anc)
    circuit.h(qs[0])
    for i in range(n):
        circuit.x(qs[i])
    for i in range(n):
        circuit.h(qs[i])

And now for the main part, finding a larger value using Grover's algorithm.

In [ ]:
def init_grover(circuit, f_input, f_output):
    for j in range(len(f_input)):
        circuit.h(f_input[j])
    circuit.x(f_output)
    circuit.h(f_output)

def grover_find_larger(n, start_value, iterations, device='local_qasm_simulator', shots=2):
    """
    return value: counts (results from running grover using 'iteration' iterations)
    """
    qc = QuantumCircuit()
    a = QuantumRegister(n, 'a')
    b = QuantumRegister(n, 'b')
    anc = QuantumRegister(2*n - 2, 'anc')
    target = QuantumRegister(1)
    cs = ClassicalRegister(n, 'result')
    
    qc.add(a)
    qc.add(b)
    qc.add(anc)
    qc.add(target)
    qc.add(cs)
    
    # init
    init_grover(qc, a, target)
    qc.barrier()
    int_to_qubit(start_value, qc, b)
    
    for t in range(iterations):
        n_bit_comparator(qc, a, b, anc, target[0])
        inv_around_mean(qc, a, anc)
    
    qc.barrier()
    qc.measure(a, cs)
    # Execute circuit
    job = execute([qc], backend=device, shots=shots)
    result = job.result()
    print(result.get_data())

    counts = result.get_counts(qc)
    counts = {int(k[::-1],2) : v for k,v in counts.items()}
    return counts

We already implemented all the components we need in order to run maximum finding algorithm.

Let's start with the simplest case, the case where every index of search space corresponds to the index value itself and we can check whether our implemenation of Grover's algorithm is correct by finding value larger than 11 (4-bit) using **one Grover's iteration**.

In [ ]:
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

    # maximum finding scenario 1: f[x] = x
    n = 11
    iterations = 0
    f_count = 0
    loop = 0
    
    counts = grover_find_larger(4, n, 1, shots=1000)
    visualization.plot_histogram(counts)

As we can see from the plots above, it seems that our individual parts are working as intended and we're ready to test running maximum finding algorithm on local simulator.

### Maximum finding algorithm on local simulator

1. initialize $m = 1$ and set $\lambda = 6/5$
2. choose `j` uniformly at random among the nonnegative integers smaller than `m`
3. apply j iterations of Grover's algorithm starting from initial state $\left|\Psi_0\right\rangle = \sum_i \frac{1}{\sqrt{N}}\left|i\right\rangle$
4. observe the register: let $i$ be the outcome
5. if `T[i] = x`, the problem is solved: **exit**
6. otherwise, set `m` to min($\lambda m$, $\sqrt{N}$)    
and go back to step 2.

In [ ]:
n = 4
total_itr = 0
f_eval = 0
cur_max_idx = 3
found = True

# loop to find larger number
while found:
    m = 1
    phi = 6/5
    itr = 0
#     print('start searching: current = {}, with j = {}, iteration count = {}'.format(cur_max_idx, j, total_itr))
    while itr <= (int(math.sqrt(2**n) / phi + 0.5) + 5):
        j = random.randrange(int(math.ceil(m)))
#         print()
        print('j = {}, m = {}'.format(j, m))
        cnts = grover_find_larger(n, cur_max_idx, j, shots=2)
        print(cnts)
        f_eval += j * 3
        itr += 1
        n_idx = max(cnts)
        m *= phi
        if n_idx > cur_max_idx:
            cur_max_idx = n_idx
            found = True
            break
    else:
        found = False

    total_itr += 3*itr

print('........')
print('search finish: max found = {}, total function evaluations = {}, total iteration count = {}'.format(cur_max_idx, f_eval, total_itr))

### Now let's test the same one with real devices.

To use the real devices, we need to register our API token to the server first.

In [ ]:
register(qx_config['APItoken'], qx_config['url'])
# available_backends({'simulator': False})

In [ ]:
import datetime

n = 4
total_itr = 0
f_eval = 0
cur_max_idx = 3
found = True

# loop to find larger number
while found:
    m = 1
    phi = 6/5
    itr = 0
    print('start searching: current = {}, iteration count = {}'.format(cur_max_idx, total_itr))
    while itr <= (int(math.sqrt(2**n) / phi + 0.5) + 5):
        j = random.randrange(int(math.ceil(m)))
        print(datetime.datetime.now())
        print('j = {}, m = {}'.format(j, m))
        cnts = grover_find_larger(n, cur_max_idx, j, shots=2, device='ibmqx5')
        print(cnts)
        f_eval += j * 3
        itr += 1
        n_idx = max(cnts)
        m *= phi
        if n_idx > cur_max_idx:
            cur_max_idx = n_idx
            found = True
            break
    else:
        found = False

    total_itr += 3*itr

print('........')
print('search finish: max found = {}, total function evaluations = {}, total iteration count = {}'.format(cur_max_idx, f_eval, total_itr))